In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage, signal, spatial, interpolate, misc
import rasterio as rio
import pandas as pd
import geopandas as gpd

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# allows zoom feature
%matplotlib inline
import mpld3
#
mpld3.enable_notebook()

plt.rcParams["figure.figsize"] = (15,15)

In [ ]:
fp_uas = r"E:\USGS\Sonoma_fires\GLA3\GLA3_2021_interp_clip_plt2.tif"

In [ ]:
with rio.open(fp_uas) as src:
    profile = src.profile

    #stacking rgb in the order of: Blue - Green - Red
    rgb = np.dstack([src.read(1).astype('int16'),src.read(2).astype('int16'),src.read(3).astype('int16')])
    #rgb = np.dstack([src.read(1),src.read(2),src.read(3)])

In [ ]:
#would masked arrays make more sense here?

shad_mask = np.empty_like(rgb)
veg_shad_mask = np.empty_like(rgb)

#Creating Excess Greeness Index 
ExG = 2*rgb[:,:,1] - rgb[:,:,2] - rgb[:,:,0]

# should try thresholded an eroded red band for the shadowmask - may correct some of the mis-classified pixels around the shadows
# especially, the rocks=

for bands in range(rgb.shape[2]):
    #threshold shadow ---- 70 was a manually expiremented value - should validate with threshold-otsu or jenks
    #threshold vegetation --- the threshold value was first obtained by jenkspy.jenks_breaks(Exg.flatten(),nb_class=2)
    shad_mask[:,:,bands] = np.where(rgb[:,:,2] < 68, np.nan, rgb[:,:,bands]) #70
    veg_shad_mask[:,:,bands] = np.where(ExG>12, np.nan, shad_mask[:,:,bands])

In [ ]:
veg_shad_mask8 = veg_shad_mask.astype('uint8')

In [ ]:
from skimage.morphology import opening, square, closing
from skimage.color import rgb2gray

grayscale = rgb2gray(veg_shad_mask8)

#opened = opening(grayscale, square(3))
closed = closing(grayscale, square(3))

In [ ]:
from skimage.segmentation import mark_boundaries, felzenszwalb
from skimage.measure import label, regionprops_table

result = felzenszwalb(closed, scale=400, sigma=0.95, min_size=3)

In [ ]:
# clear borders?

In [ ]:
fig, ax = plt.subplots(2, sharex=True, sharey=True, figsize=(20, 20))
ax[0].imshow(veg_shad_mask8)
ax[0].set_title('Pre-filter')
ax[1].imshow(mark_boundaries(closed,result))
ax[1].set_title('Post-filter')
fig.tight_layout()

In [ ]:
label_image = label(result)
props = regionprops_table(result,intensity_image=veg_shad_mask8, 
                          properties=('label','area','area_bbox','area_convex','intensity_mean','coords'))

pd.DataFrame(props).head(10)

In [ ]:
small_rocks = ((props['area'] < 600) &
                    (props['intensity_mean-2'] > 85) & #85
                    ((props['area'] / props['area_bbox']) > 0.27)) #.27

f'Found {np.count_nonzero(small_rocks == True)} rocks out of {len(small_rocks)} regions'

In [ ]:
#small_rocks and props['coords'] are the same length. Can I ZIP interate this?

rocks_classified = np.empty_like(label_image)

for i in range(len(small_rocks)):
    if small_rocks[i] == True:
        #print(i, props['coords'])
        for l in range(len(props['coords'][i])):
            x,y = props['coords'][i][l]
            rocks_classified[x,y] = 1

In [ ]:
plt.imshow(mark_boundaries(veg_shad_mask8,rocks_classified))


In [ ]:
r_0 = np.empty_like(rgb)

for bands in range(rgb.shape[2]):
    r_0[:,:,bands] = np.where(rocks_classified == 1, 0, veg_shad_mask8[:,:,bands])

In [ ]:
fig, ax = plt.subplots(2, sharex=True, sharey=True, figsize=(20, 20))
ax[0].imshow(veg_shad_mask8)
ax[0].set_title('Pre-filter')
ax[1].imshow(r_0)
ax[1].set_title('Post-filter')
fig.tight_layout()

In [ ]:
r_0_8 = r_0.astype('uint8')

In [ ]:
grayscale2 = rgb2gray(r_0_8)

opened2 = opening(grayscale, square(3))
#closed2 = closing(grayscale2, square(3))

In [ ]:

#result2 = felzenszwalb(closed2, scale=150, sigma=0.40, min_size=300)
result2 = felzenszwalb(opened2, scale=250, sigma=0.27, min_size=100) #scale 250, sigma=0.27, min_size=100

In [ ]:
fig, ax = plt.subplots(2, sharex=True, sharey=True, figsize=(20, 20))
ax[0].imshow(r_0)
ax[0].set_title('Pre-filter')
ax[1].imshow(mark_boundaries(r_0_8,result2))
ax[1].set_title('Post-filter')
fig.tight_layout()

In [ ]:
#label_image2 = label(result2)
props2 = regionprops_table(result2,intensity_image=opened2, 
                          properties=('label','area','intensity_min','intensity_mean','intensity_max','coords'))

pd.DataFrame(props2).head(10)

In [ ]:
from skimage.filters.rank import entropy
from skimage.morphology import disk

ent_img = entropy(grayscale2, disk(3))

In [ ]:
props3 = regionprops_table(result2,intensity_image=ent_img,
                            properties=('label','area','intensity_mean','coords'))

In [ ]:
pd.DataFrame(props3)

In [ ]:


large_rocks = props3['intensity_mean'] > 4.05

f'Found {np.count_nonzero(large_rocks == True)} rocks out of {len(large_rocks)} regions'

In [ ]:

rocks_classified2 = np.zeros_like(result2)

for i in range(len(large_rocks)):
    if large_rocks[i] == True:
        #print(props['label'][i])
        for l in range(len(props3['coords'][i])):
            x,y = props3['coords'][i][l]
            rocks_classified2[x,y] = 1

In [ ]:
plt.imshow(mark_boundaries(r_0_8,rocks_classified2))

In [ ]:
r_large = np.empty_like(rgb)

for bands in range(rgb.shape[2]):
    r_large[:,:,bands] = np.where(rocks_classified2 == 1, np.nan, r_0_8[:,:,bands])

In [ ]:
#finding threshold for dark / light soil

from skimage.filters import threshold_multiotsu

grayscale3 = rgb2gray(r_large)

threshold = threshold_multiotsu(grayscale3)

In [ ]:
regions = np.digitize(grayscale3, bins=threshold)

In [ ]:
np.unique(regions)

In [ ]:
fig, ax = plt.subplots(2, sharex=True, sharey=True, figsize=(20, 20))
ax[0].imshow(r_large)
ax[0].set_title('Pre-filter')
ax[1].imshow(regions)
ax[1].set_title('Post-filter')
fig.tight_layout()

## Classification

look into masked arrays 

In [ ]:
classified = np.empty_like(ExG,dtype='uint8')

    #threshold shadow ---- 70 was a manually expiremented value - should validate with threshold-otsu or jenks
    #threshold vegetation --- the threshold value was first obtained by jenkspy.jenks_breaks(Exg.flatten(),nb_class=2)
    
classified = np.where(rgb[:,:,2] < 70, 1, classified) #assigning 1 to shadows
classified = np.where(ExG>12, 2, classified) #assigning 2 to vegetation
classified = np.where(rocks_classified == 1, 3, classified)
classified = np.where(rocks_classified2 == 1, 3, classified)
classified = np.where(regions == 1, 4, classified) #dark soil
classified = np.where(regions == 2, 5, classified) #light soil
classified = np.where(classified > 5, 0, classified) #assigning unclassified pixels to 0

In [ ]:
np.unique(classified,return_counts=True)

In [ ]:
from matplotlib.colors import ListedColormap

cmap = ListedColormap(['black','black','green','grey','sienna','tan'])

In [ ]:
fig, ax = plt.subplots(2, sharex=True, sharey=True, figsize=(20, 20))
ax[0].imshow(rgb)
ax[0].set_title('Pre-filter')
ax[1].imshow(classified, cmap=cmap)
ax[1].set_title('Post-filter')
fig.tight_layout()

In [ ]:
fp_shp = r"E:\USGS\Sonoma_fires\GLA3\GLA3_2021_ctrs_32610.gpkg" #point file containing centroids of plots

In [ ]:
gdf = gpd.read_file(fp_shp)

#df['buffer'] = df.geometry.buffer(0.03) #buffer for masking plot markers
gdf['plots'] = gdf.geometry.buffer(0.48) #field plot size for training and validating
#df['sent2_plots'] = df.geometry.buffer(10, cap_style=3) #10meter plots for scaling up to sentinel2, cap_style=3 creates a square buffer

#mask = df['buffer']

In [ ]:

gdf.set_geometry("plots")

In [ ]:
shape = gdf.plots[2:3]

### need to save the classifed to memory with rasterio header then clip fieldplot 

In [ ]:
from rasterio.plot import show

with rio.open(fp_uas) as src:
    out_image, out_transform = rio.mask.mask(src, shape, crop=True)
    show(out_image)

In [ ]:
classified = classified.reshape(1,classified.shape[0],classified.shape[1])

In [ ]:
#changing the count to 1 

profile.update(
        dtype=rio.uint8,
        count=1,
        compress='lzw')

In [ ]:
#### for writing out classified file - skip this for the subsample test

outfile = fp_uas.replace('.tif','_classified.tif')

with rio.open(outfile, "w", **profile) as dest:
    dest.write(classified)

In [ ]:
from rasterio import MemoryFile

with MemoryFile() as memfile:
    with memfile.open(**profile) as dataset:
        dataset.write(classified)
 
        #print(dataset.profile)
        out_image2, out_transform2 = rio.mask.mask(dataset, shape, crop=True)
        out_meta = dataset.meta.copy()

    out_meta.update({
    "driver": "GTiff",
    "height": out_image2.shape[1],
    "width": out_image2.shape[2],
    "transform": out_transform2
    })

In [ ]:
show(out_image2,cmap=cmap)